In [1]:
# Generate Sample input code

import csv
csv_data = [
    ["Timestamp", "1", "2", "3", "4", "5", "6", "7", "8", "9", "10", "11", "12", "13", "14", "15"],
    ["2025/03/01 1:20:08 PM EST", "Ronaldo", "Ibrahimovic", "Messi", "Iniesta", "Xavi", "Modric", "Marcelo", "Buffon", "Kroos",
     "Casillas", "Carvajal", "Fabregas", "Valdes", "Roberto", "Rudiger"],
    ["2025/03/01 1:22:32 PM EST", "Messi", "Xavi", "Ronaldo", "Iniesta", "Ibrahimovic", "Modric", "Kroos", "Marcelo", "Carvajal",
     "Buffon", "Fabregas", "Casillas", "Rudiger", "Valdes", "Roberto"],
    ["2025/03/01 1:25:45 PM EST", "Xavi", "Messi", "Ronaldo", "Ibrahimovic", "Iniesta", "Modric", "Carvajal", "Kroos", "Marcelo",
     "Fabregas", "Casillas", "Buffon", "Valdes", "Rudiger", "Roberto"],
    ["2025/03/01 1:30:12 PM EST", "Ibrahimovic", "Ronaldo", "Messi", "Xavi", "Iniesta", "Modric", "Buffon", "Kroos", "Marcelo",
     "Carvajal", "Fabregas", "Casillas", "Valdes", "Rudiger", "Roberto"],
    ["2025/03/01 1:35:55 PM EST", "Iniesta", "Ibrahimovic", "Ronaldo", "Messi", "Xavi", "Modric", "Kroos", "Marcelo", "Carvajal",
     "Buffon", "Fabregas", "Casillas", "Rudiger", "Valdes", "Roberto"]
]

# Export to CSV
csv_filename = "google_form_team_ratings.csv"
with open(csv_filename, mode="w", newline="", encoding="utf-8") as file:
    writer = csv.writer(file)
    writer.writerows(csv_data)

print(f"CSV file '{csv_filename}' has been created successfully!")

# Read CSV and calculate average positions
player_positions = {}

for row in csv_data[1:]:  # Skip header
    for rank, player in enumerate(row[1:], start=1):  # Skip timestamp
        if player not in player_positions:
            player_positions[player] = []
        player_positions[player].append(rank)

# Compute average ranking for each player
average_positions = {player: sum(positions) / len(positions) for player, positions in player_positions.items()}

# Sort players by average position (lower is better)
sorted_players = sorted(average_positions.items(), key=lambda x: x[1])

# Display player rankings
print("Player Rankings by Average Position:")
for rank, (player, avg_position) in enumerate(sorted_players, start=1):
    print(f"{rank}. {player}: {avg_position:.2f}")

CSV file 'google_form_team_ratings.csv' has been created successfully!
Player Rankings by Average Position:
1. Ronaldo: 2.40
2. Messi: 2.60
3. Ibrahimovic: 2.80
4. Xavi: 3.40
5. Iniesta: 3.80
6. Modric: 6.00
7. Kroos: 7.80
8. Marcelo: 8.20
9. Carvajal: 9.20
10. Buffon: 9.40
11. Fabregas: 11.00
12. Casillas: 11.40
13. Valdes: 13.40
14. Rudiger: 13.80
15. Roberto: 14.80


**Load the results from the form**

In [5]:
import gspread
import pandas as pd
from google.colab import auth
from google.auth import default

# Authenticate and create the client
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

# Open the Google Sheet by URL
sheet_url = input("Enter the link to your sheet: ")
spreadsheet = gc.open_by_url(sheet_url)

# Select the first worksheet
worksheet = spreadsheet.sheet1

# Get all values as a list of lists
data = worksheet.get_all_values()

# Convert to Pandas DataFrame
df = pd.DataFrame(data)
spreadsheet = gc.open_by_url(sheet_url)

# Select the first worksheet
worksheet = spreadsheet.sheet1

# Get all values as a list of lists
data = worksheet.get_all_values()

# Convert to Pandas DataFrame
df = pd.DataFrame(data)

# Set the first row as headers
df.columns = df.iloc[0]
df = df[1:].reset_index(drop=True)

csv_filename = "google_form_team_ratings.csv"
df.to_csv(csv_filename, index=False)

# Display confirmation
print(f"CSV file saved as '{csv_filename}'")

Enter the link to your sheet: https://docs.google.com/spreadsheets/d/1BpaRVx2sb0f1JzDD8wHj4E-nAVv50D22prniRl-_Icc/edit?usp=sharing
CSV file saved as 'google_form_team_ratings.csv'


**Generate Teams**

In [4]:
import csv
import random
from collections import defaultdict, deque

def read_csv_file(file_path):
    """Reads a single CSV file and extracts player rankings."""
    player_scores = defaultdict(list)

    with open(file_path, newline='', encoding='utf-8') as f:
        reader = csv.DictReader(f)
        for row in reader:
            for rank, player in row.items():
                if rank.isdigit() and player:  # Ensure it's a valid ranking column
                    player_scores[player].append(int(rank))

    return player_scores

def calculate_ratings(player_scores):
    """Assigns a rating from 1 to 5 based on sorted rankings."""
    sorted_players = sorted(player_scores.items(), key=lambda x: sum(x[1]) / len(x[1]))
    num_players = len(sorted_players)

    ratings = {}
    for i, (player, _) in enumerate(sorted_players):
        if i < num_players * 0.2:
            ratings[player] = 5
        elif i < num_players * 0.4:
            ratings[player] = 4
        elif i < num_players * 0.6:
            ratings[player] = 3
        elif i < num_players * 0.8:
            ratings[player] = 2
        else:
            ratings[player] = 1

    return ratings

def balance_teams(ratings, num_teams, absent_players, goalkeepers):
    """Distributes available players into balanced teams while ensuring randomness."""
    available_players = {player: rating for player, rating in ratings.items() if player not in absent_players}
    # check if number of teams is divisible by the number of available players, if not get the extra players we need
    guest_players = []
    if len(available_players) % num_teams != 0:
        extra_players = num_teams - (len(available_players) % num_teams)
        for i in range(extra_players):
            guest_players.append(f"Guest Player {i+1}")

    # get the maximum number of players for each team
    max_players_per_team = len(available_players) // num_teams

    # remove goalkeepers from the players list
    for player in goalkeepers:
        if player in available_players:
            del available_players[player]

    sorted_players = sorted(available_players.items(), key=lambda x: -x[1])  # Sort by skill level (higher first)

    teams = [[] for _ in range(num_teams)]

    # Distribute players ensuring skill balance in a 2-d list
    skill_buckets = [[] for _ in range(5)]
    for player, rating in sorted_players:
      #  combine the last 2 skill buckets for more randomness
      if rating <= 2:
        skill_buckets[1].append(player)
      else:
        skill_buckets[rating - 1].append(player)

    for i in range(4,0,-1):
        # while there are still players
        while skill_buckets[i]:
                random_player = random.choice(skill_buckets[i])
                # add the player to the team with least players that did not exceed max_players_per_team
                min_team = min(teams, key=len)
                if len(min_team) < max_players_per_team:
                    min_team.append(random_player)
                    skill_buckets[i].remove(random_player)

    # add goalkeepers to the teams with least players
    for i, goalkeeper in enumerate(goalkeepers):
        min_team = min(teams, key=len)
        min_team.append(goalkeeper)

    # add guest players to the teams with least players
    for i, player in enumerate(guest_players):
        min_team = min(teams, key=len)
        min_team.append(player)

    # Shuffle teams to add randomness
    for team in teams:
        random.shuffle(team)

    return teams

def display_top_players(ratings):
    """Displays the top X players based on user input."""
    num_top_players = int(input("Enter the number of top players to display: "))
    sorted_players = sorted(ratings.items(), key=lambda x: -x[1])[:num_top_players]

    print("\nTop Players:")
    for i, (player, rating) in enumerate(sorted_players, 1):
        print(f"{i}. {player}: {rating}")

def main():
    # file_path = input("Enter the path to the CSV file: ")
    player_scores = read_csv_file(csv_filename)

    if not player_scores:
        print("No valid player data found.")
        return

    absent_players = input("Enter absent players separated by commas: ").split(',')
    absent_players = {player.strip() for player in absent_players}

    num_teams = int(input("Enter the number of teams: "))

    goalkeepers = input("Enter goalkeepers separated by commas (up to the number of teams): ").split(',')
    goalkeepers = [gk.strip() for gk in goalkeepers if gk.strip()][:num_teams]  # Ensure max goalkeepers = num_teams

    ratings = calculate_ratings(player_scores)
    teams = balance_teams(ratings, num_teams, absent_players, goalkeepers)

    # Display teams
    for i, team in enumerate(teams, 1):
        print(f"Team {i}: {', '.join(team)}")

    # Display top X players
    display_top_players(ratings)

if __name__ == "__main__":
    random.seed()  # Ensure randomness for different runs
    main()

Enter absent players separated by commas: Carvajal
Enter the number of teams: 3
Enter goalkeepers separated by commas (up to the number of teams): Valdes,Casillas,Buffon
Team 1: Ronaldo, Modric, Casillas, Roberto, Marcelo
Team 2: Iniesta, Buffon, Messi, Fabregas, Kroos
Team 3: Xavi, Valdes, Rudiger, Ibrahimovic, Guest Player 1
Enter the number of top players to display: 5

Top Players:
1. Ronaldo: 5
2. Messi: 5
3. Ibrahimovic: 5
4. Xavi: 4
5. Iniesta: 4
